# \begin{align}
    \color{yellow}{\textbf{Proyecto Telecom X Latam: Predicción de Cancelación}}
\end{align}

$\color{yellow}{\textbf{Sobre el proyecto:}}$

<p>El desafío <strong>Telecom X Latam-parte 2</strong> forma parte de la formación de Data Science del programa ONE, especificamente de la ruta Machine Learning.</p>

<p>La empresa Telecom X está enfrentando una alta tasa de cancelaciones y en una primera instancia, como parte del equipo de análisis de datos, se realizó el análisis exploratorio para detectar los factores que están llevando a los usuarios a dejar la empresa.<br>

Una vez indentificados patrones o tendencias que llevan a las personas a cancelar los servicios de la empresa, como parte del equipo de Machine Learning, desarrollamos modelos de clasificación:<strong> Árbol de Decisión y Regresión logística</strong>, capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelación, y le corresponde al equipo construir un pipeline robusto para esta etapa inicial de modelado.</p>



$\color{yellow}{\textbf{Objetivos del desafío:}}$

<ul>
<li>Preparar los datos para el modelado (tratamiento, codificación, normalización).</li>

<li>Realizar análisis de correlación y selección de variables.
Entrenar dos o más modelos de clasificación.</li>

<li>Evaluar el rendimiento de los modelos con métricas.</li>

<li>Interpretar los resultados, incluyendo la importancia de las variables.</li>

<li>Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación.</li>
</ul>

$\color{yellow}{\textbf{Prácticas del desafío:}}$

<ul>
<li> Preprocesamiento de datos para Machine Learning.</li>
<li> Construcción y evaluación de modelos predictivos.</li>
<li> Interpretación de resultados y entrega de insights.</li>
<li> Comunicación técnica con enfoque estratégico.</li>
</u>

## $\color{yellow}{\textbf{Preprocesamiento de los datos:}}$

In [ ]:
#Importamos bibliotecas para manipular y visualizar los datos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
#Lectura del df limpio obtenido en el desafío anterior
ruta = '/content/drive/MyDrive/Colab Notebooks/datos_limpios.csv'
clientes = pd.read_csv(ruta)

In [ ]:
clientes.head()

In [ ]:
#Eliminamos las columnas que no tienen relevancia para el modelo
clientes.drop(columns=['customerID','Gender','CuentasDiarias'],
              inplace=True)

In [ ]:
#Eliminamos los valores NaN de la columna ChargesTotal
clientes.dropna(inplace=True)
clientes.info()

In [ ]:
#Identificamos las columnas categóricas
columnas_categoricas = clientes.select_dtypes(include='object').columns
columnas_categoricas

In [ ]:
#Función para obtener los valores únicos de cada columna categórica
def valores_unicos(df):
  columnas_categoricas = df.select_dtypes(include='object').columns
  for columna in columnas_categoricas:
    print(f'{columna}: {df[columna].unique()}')

In [ ]:
valores_unicos(clientes)

In [ ]:
#Llamamos la función valores_unicos para ver los valores únicos del df clientes
clientes[['MultipleLines','InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies']] = clientes[['MultipleLines','InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies']].replace({'no internet service':'no', 'no phone service':'no'})

valores_unicos(clientes)

### $\color{yellow}{\textbf{Analizando la relación entre variables categóricas}}$

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
"""
Función que calcula el coeficiente de Cramer
Utilizamos el coeficiente corregido de V-Cramer, ya que el no corregido puede
introducir sesgos
"""
def v_cramer_corregido(col1, col2):
  tabla_cont = pd.crosstab(col1, col2)
  estadistico_chi2 = chi2_contingency(tabla_cont)[0]
  n = tabla_cont.sum().sum()
  estadistico_phi2 = estadistico_chi2 / n
  renglon, columnas = tabla_cont.shape
  phi2corr = max(0, estadistico_phi2 - ((columnas-1)*(renglon-1))/(n-1))
  renglon_corr = renglon - ((renglon-1)**2)/(n-1)
  columnas_corr = columnas - ((columnas-1)**2)/(n-1)
  return np.sqrt(phi2corr / min((columnas_corr-1), (renglon_corr-1)))

In [ ]:
#Creamos una variable con los nombres de las columnas categóricas
variables_cat = ['Churn', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod']
variables_cat

In [ ]:
#Creamos la matriz que guardará los coeficientes de V de Cramer
v_cramer_matrix = pd.DataFrame(index=variables_cat, columns=variables_cat)
v_cramer_matrix

In [ ]:
#Guardamos en la matriz anterior el coeficiente para cada par de variables categóricas
for col1 in variables_cat:
    for col2 in variables_cat:
      v_cramer_matrix.loc[col1, col2] = v_cramer_corregido(clientes[col1], clientes[col2])
v_cramer_matrix

In [ ]:
#Gráfico de la matriz de Cramer
plt.figure(figsize=(12, 10))
sns.heatmap(v_cramer_matrix.astype(float), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de V de Cramer para Variables Categóricas")
plt.show()

In [ ]:
#Filtro para elegir las variables categóricas que tienen una relación más fuerte con Churn
target_var = 'Churn'
v_relevantes = v_cramer_matrix.index[abs(v_cramer_matrix[target_var]) >= 0.20].tolist()
v_relevantes

In [ ]:
"""
Creamos otra matriz con solo los coeficientes de las variables
más fuertemente relacionadas con Churn
"""
v_filtrada = v_cramer_matrix.loc[v_relevantes, v_relevantes]
v_filtrada

In [ ]:
"""
Creamos una máscara booleana para ocultar el triángulo superior
de la matriz anterior
"""
mascara = np.triu(np.ones_like(v_filtrada, dtype=bool))

In [ ]:
#Gráfico de la matriz filtrada
plt.figure(figsize=(12, 10))
sns.heatmap(v_filtrada.astype(float), annot=True, cmap='coolwarm', fmt=".2f", mask=mascara)
plt.title("Matriz de V de Cramer para Variables Categóricas Relevantes")
plt.show()

### $\color{yellow}{\textbf{Correlación entre variables numéricas y con churn}}$


In [ ]:
"""
Seleccionamos las variables numéricas y luego calculamos la correlación entre
las variables númericas más la variable objetivo churn
"""
variables_numericas = clientes.select_dtypes(include=['number'])
variables_numericas['Churn'] = clientes['Churn'].replace({'no': 0, 'yes': 1})
print('\t\tMatriz de Correlación entre variables numéricas independientes y churn\t\t')
print(variables_numericas.drop(columns='SeniorCitizen', axis=1).corr())

In [ ]:
#Distribución de Tenure según si los clientes hiceron o no churn
fig, ax = plt.subplots(figsize=(10,6))
ax = sns.boxplot(data=variables_numericas, x='Tenure', hue='Churn', palette='PiYG_r')
ax.set_title('Distribución de Antiguedad y Cancelación')
ax.set_xlabel('Antiguedad')
ax.set_xlim(0,80)
ax.legend(title="Cancelación",labels=['No', 'Sí'], loc='best',  bbox_to_anchor=(1,0.70,0.15,0.3))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax = sns.histplot(variables_numericas,x='Tenure', hue='Churn', kde=True, stat='frequency')
ax.set_title('Histograma de Antiguedad y Cancelación')
ax.set_xlabel('Antiguedad')
ax.set_ylabel('Frecuencia')
ax.set_xlim(0,70)
ax.set_ylim(0,150)
ax.legend(title="Cancelación",labels=['Sí', 'No'], loc='upper right', bbox_to_anchor=(1,0.70,0.15,0.3))

plt.show()


In [ ]:
#Distribución de cargos mensuales según si los clientes hiceron o no churn
fig, ax = plt.subplots(figsize=(10,6))
ax = sns.boxplot(data=variables_numericas, x='ChargesMonthly', hue='Churn', palette='PiYG_r')
ax.set_title('Distribución de Cargos mensuales y Cancelación')
ax.set_xlabel('Cargos mensuales')
ax.set_xlim(0,120)
ax.legend(title="Cancelación",labels=['No', 'Sí'], loc='upper right', bbox_to_anchor=(1,0.70,0.15,0.3))

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.histplot(variables_numericas,x='ChargesMonthly', hue='Churn',kde=True, stat='frequency')
ax.set_title('Histograma de Cargos mensuales y Cancelación')
ax.set_xlabel('Cargos mensuales')
ax.set_ylabel('Frecuencia')
ax.set_xlim(20,120)
ax.set_ylim(0,160)
ax.legend(title="Cancelación",labels=['Sí', 'No'], loc='upper right', bbox_to_anchor=(1,0.70,0.15,0.3))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax = sns.boxplot(data=variables_numericas, x='ChargesTotal', hue='Churn', palette='PiYG_r')
ax.set_title('Distribución de Cargos totales y Cancelación')
ax.set_xlabel('Cargos totales')
ax.set_xlim(0,8500)
ax.legend(title="Cancelación",labels=['No', 'Sí'], loc='upper right', bbox_to_anchor=(1,0.70,0.15,0.3))

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.histplot(variables_numericas,x='ChargesTotal', hue='Churn',kde=True, stat='frequency')
ax.set_title('Histograma de Cargos totales y Cancelación')
ax.set_xlabel('Cargos totales')
ax.set_ylabel('Frecuencia')
ax.set_xlim(0,9000)
ax.set_ylim(0,2)
ax.legend(title="Cancelación",labels=['Sí', 'No'], loc='upper right', bbox_to_anchor=(1,0.70,0.15,0.3))
plt.show()

### $\color{yellow}{\textbf{Análisis de valores atípícos en Tenure y ChargesTotal}}$

In [ ]:
import numpy as np

In [ ]:
#Filtro para separar los clientes que han cancelado
churn = variables_numericas['Churn'] == 1
#Creamos un dataframe con las variables que presentan datos atípicos
df_atipicos = variables_numericas[['Tenure', 'ChargesTotal', 'Churn']]
#Acá aplicamos el filtro anterior para quedarnos con los datos de los clientes que han hecho churn
df_atipicos = df_atipicos[churn]


In [ ]:
variables_numericas['Churn'].value_counts()

In [ ]:
indices_eliminar = []

In [ ]:
#Definimos los límites superior e inferior para identificar los valores atípicos en Tenure
q1, q3 = np.percentile(df_atipicos['Tenure'], [25,75])
ric = q3 - q1
ten_inferior = q1 - (1.5 * ric)
ten_superior = q3 + (1.5 * ric)
#Guaramos los indices de los valores atípicos identificados
indice_atipicos_ten = df_atipicos[(df_atipicos['Tenure'] < ten_inferior) | (df_atipicos['Tenure'] > ten_superior)].index
indice_atipicos_ten

In [ ]:
#Los indices los guardAmos en la lista indices_eliminar
indices_eliminar.extend(indice_atipicos_ten)

In [ ]:
#Contamos la cantidad de datos a eliminar
len(indices_eliminar)

In [ ]:
#Definimos los límites superior e inferior para identificar los valores atípicos en Charges total
q1, q3 = np.percentile(df_atipicos['ChargesTotal'], [25,75])
ric = q3 - q1
charges_inferior = q1 - (1.5 * ric)
charges_superior = q3 + (1.5 * ric)
#Guaramos los indices de los valores atípicos identificados
indice_atipicos_char = df_atipicos[(df_atipicos['ChargesTotal'] < charges_inferior) | (df_atipicos['ChargesTotal'] > charges_superior)].index
len(indice_atipicos_char)

In [ ]:
#Los indices los guardmos en la lista indices_eliminar
indices_eliminar.extend(indice_atipicos_char)

In [ ]:
#Contamos la cantidad de datos a eliminar
len(indices_eliminar)

In [ ]:
#Nos aseguramos que los indices no se repitan
indices_eliminar = list(set(indices_eliminar))
len(indices_eliminar)

In [ ]:
#Eliminamos los indices de los valores atípicos del df variables_numericas y
#guardamos los resultados en otro df llamado df_limpio
df_limpio = variables_numericas.drop(indices_eliminar)

In [ ]:
df_limpio['Churn'].value_counts()

In [ ]:
df_limpio.columns

In [ ]:
from scipy import stats

In [ ]:
"""
Calculo de coeficiente de correlación biserial entre la variable Churn y las variables numéricas
para determinar la correlación entre las variables numéricas  y la variable categórica churn
después de la eliminación de los valores atípicos
"""
columnas = df_limpio.columns.drop(['SeniorCitizen', 'Churn'])
coeficiente_biserial = []
for columna in columnas:
  coeficiente = stats.pointbiserialr(df_limpio[columna], df_limpio['Churn'])
  coeficiente_biserial.append((columna,coeficiente[0]))
coeficiente_biserial

In [ ]:
"""
Verificación de la distribución de Tenure y Churn después de elimnar los
valores atípicos
"""
sns.boxplot(data=df_limpio, x='Tenure', hue='Churn')

In [ ]:
"""
Verificación de la distribución de Tenure y Churn después de elimnar los
valores atípicos
"""
sns.histplot(df_limpio,x='Tenure', hue='Churn',kde=True, stat='frequency')

In [ ]:
"""
Verificación de la distribución de Charges total y Churn después de elimnar los
valores atípicos
"""
sns.boxplot(data=df_limpio, x='ChargesTotal', hue='Churn')

In [ ]:
"""
Verificación de la distribución de Charges total y Churn después de elimnar los
valores atípicos
"""
sns.histplot(df_limpio,x='ChargesTotal', hue='Churn',kde=True, stat='frequency')

In [ ]:
"""
Correlación entre las variables numéricas despúes de eliminar
los valores atípicos
"""
df_limpio[['Tenure', 'ChargesMonthly','ChargesTotal']].corr()

### $\color{yellow}{\textbf{Dividiendo entre variables explicativas y objetivo}}$

In [ ]:
X = clientes.drop(['Churn', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                   'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport',
                   'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'ChargesTotal'], axis=1)
y = clientes.Churn

### $\color{yellow}{\textbf{Codificación de las variables categóricas independientes}}$

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
columnas = X.columns

In [ ]:
one_hot = make_column_transformer((OneHotEncoder(drop='first'),
                                   ['SeniorCitizen', 'InternetService',
                                    'Contract', 'PaymentMethod']),
                                    remainder='passthrough',
                                    sparse_threshold=0,
                                    force_int_remainder_cols=False)

one_hot

In [ ]:
X = one_hot.fit_transform(X)
X

In [ ]:
one_hot.get_feature_names_out(columnas)

In [ ]:
one_hot.get_feature_names_out(columnas)

In [ ]:
df_codificado = pd.DataFrame(X, columns= one_hot.get_feature_names_out(columnas))

In [ ]:
df_codificado.head()

### $\color{yellow}{\textbf{Codificando variable objetivo}}$

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [ ]:
y

### $\color{yellow}{\textbf{Analizando la proporción de cancelación para encontrar desbalanceo}}$

In [ ]:
clientes.Churn.value_counts(normalize=True)

In [ ]:
clientes.info()

##$\color{yellow}{\textbf{Modelado predictivo}}$

###$\color{yellow}{\textbf{División de los datos en entrenamiento, validación y prueba}}$

In [ ]:
#Importamos el módulo para dividir los datos
from sklearn.model_selection import train_test_split

In [ ]:
# Dividimos los datos en entrenamiento y prueba
X, X_test, y, y_test = train_test_split(X,y, test_size=0.30, stratify=y, random_state=42)

In [ ]:
#Dividimos los datos de entrenamiento, uno para el aprendizaje y otro para validar las generalizaciones del modelo
X_train, X_val, y_train, y_val = train_test_split(X,y, stratify=y, random_state=42)

###$\color{yellow}{\textbf{Normalización y estandarización  de los datos}}$

In [ ]:
#Importamos RobusScaler para el escalamiento de los datos, ya que es robusto a los datoa atípicos
# y no tiene el supuesto de que los datos tienen una distribución normal
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()

X_train = robust_scaler.fit_transform(X_train)
X_val = robust_scaler.transform(X_val)
X_test = robust_scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
df_normalizado = pd.DataFrame(X_train,columns= one_hot.get_feature_names_out(columnas))
df_normalizado.info()

###$\color{yellow}{\textbf{Análisis de multicolinealidad entre las variables del modelo}}$

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
#Realizamos el análisis de  multicolinealidad en los datos de entrenamiento ya normalizados
df = df_normalizado.copy()
vif_data = pd.DataFrame()
vif_data['Feature'] = df.columns
vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif_data

In [ ]:
"""
Descartamos la variable ChargesMonthly ya que tiene un vif >= 5
Descartamos la variable Internet Service no ya que se relaciona con internet Service fiber optic
Nos quedamos con las variables asociadas con el método de pago ya que el valor de vif es bajo
Lo mismo sucede con las variables relacionadas con el tipo de contrato
"""
df = df.drop(columns=['onehotencoder__InternetService_no', 'remainder__ChargesMonthly'])
vif_data = pd.DataFrame()
vif_data['Feature'] = df.columns
vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif_data

In [ ]:
#Borramos las columnas mencionadas anteriormente
_X_train = np.delete(X_train, [2,9], axis=1)
_X_val = np.delete(X_val, [2,9], axis=1)
_X_test = np.delete(X_test, [2,9], axis=1)

###$\color{yellow}{\textbf{Balanceo de los datos}}$

In [ ]:
"""
Se balancea los datos de entrenamiento,
dado que detectamos un desbalanceo en las clases de churn
"""
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(_X_train, y_train)

##$\color{yellow}{\textbf{Entrenamiento de los modelos}}$

###$\color{yellow}{\textbf{Modelo dummy}}$

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
modelo_base = DummyClassifier()

In [ ]:
#Entrenamiento del modelo base
modelo_base.fit(X_train_bal, y_train_bal)

In [ ]:
print(f'La exactidud del modelo de entrenamiento es {modelo_base.score(X_train_bal,y_train_bal)}')

###$\color{yellow}{\textbf{Árbol de Decisión}}$

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
modelo_arbol = DecisionTreeClassifier(max_depth=5)
#Entrenamiento del modelo árbol
modelo_arbol.fit(X_train_bal, y_train_bal)

In [ ]:
#Métrica del modelo de entrenamiento
print(f'La exactidud del modelo de entrenamiento es {modelo_arbol.score(X_train_bal,y_train_bal)}')

In [ ]:
##Métrica del modelo con los datos de validación
print(f'La exactidud del modelo de validación es {modelo_arbol.score(_X_val,y_val)}')

### **Modelo Regresión Logística**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
modelo_regresion = LogisticRegression(random_state=42)
#Entrenamiento del modelo de regresión logística
modelo_regresion.fit(X_train_bal, y_train_bal)

In [ ]:
#Métrica del modelo de entrenamiento
print(f'La exactidud del modelo de entrenamiento es {modelo_regresion.score(X_train_bal,y_train_bal)}')

###$\color{yellow}{\textbf{Evaluación de los modelos}}$

In [ ]:
#Importamos las métricas para evaluar los modelos
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

####$\color{yellow}{\textbf{Evaluación del modelo Árbol de Decisión}}$

In [ ]:
y_predicho = modelo_arbol.predict(_X_val)

matriz_confusion = confusion_matrix(y_val, y_predicho)

print(matriz_confusion)

In [ ]:
vis_matriz_confusion = ConfusionMatrixDisplay(matriz_confusion)

vis_matriz_confusion.plot();

#####$\color{yellow}{\textbf{Informe de métricas}}$

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_val, y_predicho))

#####$\color{yellow}{\textbf{Análisis de curvas ROC y Precision x Recall}}$

In [ ]:
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score

In [ ]:
RocCurveDisplay.from_predictions(y_val, y_predicho, name='Árbol de Decisión');

In [ ]:
print(f'El área debajo de la curva ROC es {roc_auc_score(y_val, y_predicho)}')

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay, average_precision_score

In [ ]:
PrecisionRecallDisplay.from_predictions(y_val, y_predicho);

In [ ]:
print(f'El área debajo de la curva Precision vs Recall es {average_precision_score(y_val, y_predicho)}')

####$\color{yellow}{\textbf{Evaluación del modelo Regresión logística}}$

In [ ]:
y_regresion_predicho = modelo_regresion.predict(_X_val)

####$\color{yellow}{\textbf{Reporte de métricas}}$

In [ ]:
print(classification_report(y_val, y_regresion_predicho))

In [ ]:
matriz_confusion_regresion = confusion_matrix(y_val, y_regresion_predicho)

In [ ]:
vis_matriz_confusion_regresion = ConfusionMatrixDisplay(matriz_confusion_regresion)

vis_matriz_confusion_regresion.plot();


#####$\color{yellow}{\textbf{Análisis de curvas ROC y Precision x Recall}}$

In [ ]:
RocCurveDisplay.from_predictions(y_val, y_regresion_predicho, name='Regresión Logistica');

In [ ]:
print(f'El área debajo de la curva ROC es {roc_auc_score(y_val, y_regresion_predicho)}')

In [ ]:
PrecisionRecallDisplay.from_predictions(y_val, y_regresion_predicho);

In [ ]:
print(f'El área debajo de la curva Precision vs Recall es {average_precision_score(y_val, y_regresion_predicho)}')

#####$\color{yellow}{\textbf{Evaluación de los modelos con validación cruzada a través de un pipeline}}$

In [ ]:
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
clientes.columns

In [ ]:
_X = clientes.drop(columns=['Churn','Partner', 'Dependents',
                            'PhoneService', 'MultipleLines', 'OnlineSecurity',
                            'OnlineBackup', 'DeviceProtection',
                            'TechSupport', 'StreamingTV',
                            'StreamingMovies', 'PaperlessBilling',
                            'ChargesMonthly', 'ChargesTotal'], axis=1)
_y = clientes.Churn

In [ ]:
Y = label_encoder.fit_transform(_y)


In [ ]:
_X.columns

In [ ]:
var_categoricas = ['SeniorCitizen', 'InternetService', 'Contract', 'PaymentMethod']
var_numericas = ['Tenure']

In [ ]:
#Instanciamos los modelos
arbol = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')

regresion = LogisticRegression(random_state=42, class_weight='balanced')

In [ ]:
#Preprocesador es la variable que contiene los pasos de preprocesamiento a seguir
preprocesador = ColumnTransformer(
                    transformers=[
                        ('num', RobustScaler(), var_numericas),
                        ('cat', OneHotEncoder(drop='first'), var_categoricas)],
                    remainder='passthrough'
                    )

In [ ]:
#Creamos el pipeline para el árbol de decisión
pipeline = imbpipeline(steps=[('preprocesador', preprocesador),
                                ('muestreo',SMOTE(random_state=42)),
                                ('árbol', arbol)])


In [ ]:
#Creamos el pipeline para la regresión logística
pipeline_regresion = imbpipeline(steps=[('preprocesador', preprocesador),
                                ('muestreo',SMOTE(random_state=42)),
                                ('regresión logistica', regresion)])

In [ ]:
#Creamos el objeto que se encarga de la validación cruzada
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
#Aplicamos la validación cruzada para el modelo árbol
cross_validation_arbol = cross_val_score(pipeline, _X, Y, cv=skfold, scoring='recall')

In [ ]:
cross_val_arbol = cross_validate(pipeline, _X, Y, cv=skfold, scoring=['precision','recall'])

In [ ]:
print(f'{cross_val_arbol["test_precision"].mean()} ± {cross_val_arbol["test_precision"].std()}')

In [ ]:
print(f'{cross_val_arbol["test_recall"].mean()} ± {cross_val_arbol["test_recall"].std()}')

In [ ]:
#Obtenemos el promedio de la métrica precisión de las 5 iteraciones
print(f'{cross_validation_arbol.mean()} +- {cross_validation_arbol.std()}')

In [ ]:
#Aplicamos la validación cruzada para el regresión logística
cross_validation_regresion = cross_val_score(pipeline_regresion, _X, Y, cv=skfold, scoring='precision')

In [ ]:
#Obtenemos el promedio de la métrica precisión de las 5 iteraciones
print(f'{cross_validation_regresion.mean()} +- {cross_validation_regresion.std()}')

In [ ]:
cross_val_regresion = cross_validate(pipeline_regresion, _X, Y, cv=skfold, scoring=['precision','recall'])

In [ ]:
print(f'{cross_val_regresion["test_precision"].mean()} ± {cross_val_regresion["test_precision"].std()}')

In [ ]:
print(f'{cross_val_regresion["test_recall"].mean()} ± {cross_val_regresion["test_recall"].std()}')

#####$\color{yellow}{\textbf{Prueba del modelo Árbol de Decisión con datos de prueba}}$

In [ ]:
y_prueba_predicho = modelo_arbol.predict(_X_test)

In [ ]:
print(classification_report(y_test, y_prueba_predicho))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_prueba_predicho);

#####$\color{yellow}{\textbf{Prueba del modelo Regresión Logística con datos de prueba}}$

In [ ]:
y_prueba_predicho = modelo_regresion.predict(_X_test)

In [ ]:
print(classification_report(y_test, y_prueba_predicho))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_prueba_predicho);

#####$\color{yellow}{\textbf{Análisis de importancia de las variables de la regresión}}$

In [ ]:
#Coeficientes del modelo de regresión logistica
modelo_regresion.coef_

In [ ]:
df.columns

In [ ]:
#Intercepto del modelo de regresión logistica
modelo_regresion.intercept_

In [ ]:
from sklearn.inspection import permutation_importance
results = permutation_importance(modelo_regresion, _X_test, y_test, scoring='recall')
for i, v in enumerate(results.importances_mean):
    print(f'Feature: {i}, Score: {v *100:.5f}')

#####$\color{yellow}{\textbf{Análisis de importancia de las variables del Árbol de Decisión}}$

In [ ]:
importances = modelo_arbol.feature_importances_ * 100
features = ['SeniorCitizen', 'Fiber optic', 'Contract_one-year','Contract_two-year',
            'PM credit card','PMelectronic check','PM mailed check','Tenure']

for i in range(len(importances)):
  print(f'Feature: {i}, Score: {importances[i]}')


In [ ]:
importances = np.sort(importances)[::-1]
importances

In [ ]:
dic_importances = {
                    'feature': ['Contract_two-year','Contract_one-year',
                                'Fiber optic','Tenure', 'PMelectronic check',
                                'SeniorCitizen', 'PM credit card','PM mailed check'],
                    'importance': importances
                  }

In [ ]:
df_importances = pd.DataFrame(dic_importances)
df_importances

In [ ]:
df_importances.info()

In [ ]:
fig, ax = plt.subplots()
ax = sns.barplot(data=df_importances, x='importance', y = 'feature', orient='h')
ax.set_title('Árbol de Decisión: Importancia de las variables')
ax.set_xlabel('Importancia(%)')
ax.set_ylabel('Característica')


## \begin{align}
    \color{yellow}{\textbf{📄Informe de predicción de cancelación}}
    \end{align}

### \begin{align}
    \color{yellow}{\textbf{Introducción}}
    \end{align}

La empresa <strong>Telecom X Latinoamérica</strong> está presentando una alta tasa de cancelación, es por eso que ha encargado a su equipo de análisis de datos y de Machine Learning dos tareas distintas pero complementarias: a partir del análisis exploratorio de los datos de los clientes identificar los factores que están influyendo en la baja de los servicios y la contrucción de modelos de clasificación que sean capaces de predecir la probabilidad de que un cliente renuncie.

En el reporte exploratorio de los datos, <a href="https://github.com/JGarcia575/challenge_telecomX.git" >ver informe</a>, se destacan:
<ul>
  <li>Edad.</li>
  <li>Tipo de contrato.</li>
  <li>Antiguedad.</li>
  <li>El servicio de internet.</li>
  <li>El cargo mensual.</li>
  <li>El cargo total.</li>
</ul>

entre los clientes que han disertado.

A partir de los datos ya tratados por el equipo de análisis de datos y apoyándonos en los hallazgos encontrados en el informe anterior, se construiran modelos predictivos de clasificación que tengan la capacidad de predecir si un cliente abandonará los servicios de la empresa. El rendimiento de los modelos serán evaluados y se eligirá el que tiene mejor rendimiento a partir de los resultados de las métricas. Por último, se proponen estrategias de retención basadas en los resultados obtenidos.



### \begin{align}
    \color{yellow}{\textbf{Métodología}}
    \end{align}

Los datos fueron preprocesados para que los algoritmos de clasificación de Machine Learning fueran capaces de interpretarlos correctamente.
El preprocesamiento incluyó:
<ul>
  <li>Eliminación de valores nulos.</li>
  <li>Codificación de las variables categóricas.</li>
  <li>Normalización de las variables numéricas.</li>
  <li>Análisis de correlación de las variables.</li>
  <li>Balanceo de datos.</li>
</ul>

Luego del procesamiento, se entrenaron dos tipos de modelos: Árbol de Decisión y Regresión Logística. Para evaluar que tan bien generalizó el modelo y para ajustar los hiperparámetros se realizó la validación del modelo utilizando una parte de los datos y, también, a través de la validación cruzada.

Por último, se realizó la evaluación final de los modelos utilizando los datos de prueba y se analizó la importancia de las variables para cada modelo.


### \begin{align}
    \color{yellow}{\textbf{Resultados}}
    \end{align}

### _<strong>Evaluación con datos de entrenamiento</strong>_
En el entrenamiento los modelos obtuvieron los siguientos puntajes de exactitud(accuracy): 0.75 para el árbol de decisión y 0.77 para el modelo de regresión, lo cual nos indica que ambos no están sobreajustando los datos.
### _<strong>Evaluación con datos de validación</strong>_
En la tabla 1, se muestra los resultados de la validación para las métricas precision, recall y f1 para el árbol de decisión.


<table>
    <thead>
        <tr>
            <th align='center'>Modelo: Árbol</th>
            <th colspan=4 align='center'>Métricas</tr>            
        </tr>
    </thead>
    <thead>
        <tr>
            <th>Clase</th>
            <th>Precision</th>
            <th>Recall</th>
            <th>F1</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td align="center">No</td>
            <td  align="center">0.90</td>
            <td  align="center">0.73</td>
            <td  align="center">0.81</td>                        
        </tr>
        <tr>
            <td align="center">Si</td>
            <td  align="center">0.51</td>
            <td  align="center">0.71</td>
            <td  align="center"> 0.62 </td>            
        </tr>        
    </tbody>
</table>

La métricas muestran que el árbol de decisión puede identificar correctamente los que hicieron churn(70% de sensibilidad). Sin embargo, solo el 51% de las predicciones de churn hechas por el modelo fueron correctas, lo que se refleja en la métrica de precisión.

La matriz de confusión ayuda a avisualizar lo que las métricas están indicando:

<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/matriz_confusion_arbol.png'>

En la matriz se observa bien que tiene una alta tasa de verdaderos positivos(255) y una baja tasa de falsos negativos(72), lo que se refleja en la métrica sesibilidad( o recall). Sin embargo, tiene una alta tasa de falsos positivos(243), similar a la tasa de verdederos positivos(255), lo cual hace que la precisión sea de aproximadamente 50%.

La curva ROC indaga sobre la capacidad del modelo para distinguir las clases, mostrando el balance entre la tasa de verdaderos positivos y la tasa falsos positivos. El modelo tiene un de área debajo de la curva del 0.76,puntuación AUC,esto significa que el árbol de decisión es capaz de distinguir entre ambas clases de clientes. No obstante, en el gráfico se observa que a medida que aumenta los verdaderos positivos, también aumenta los falsos positivos y esto implica que el modelo predice más clientes que cancelaron a costa de identificar erróneamente usuarios que no se fueron.

<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/auc_arbol.png' >

El árbol tiene una limitación y esta se observa en el gráfico de precisión vs sensibilidad. Si observamos este gráfico vemos que el mejor recall que podemos obtener es de aproximadamente 0.78 y la mejor precisión es del 0.52. Si quisieramos aumentar la métrica a 1, la precisión baja drásticamente, y si quisieramos aumentar la precisión, la sensibilidad disminuye también drásticamente. El mejor equibibrio entre sensibilidad y precisión es la combinación que se mencionó anteriormente. Por lo tanto, esto refleja una limitación del modelo.

<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/pr_arbol.png'>

La tabla 2 muestra las métricas obtenidas para el modelo de regresión con los datos de validación.

<table>
    <thead>
        <tr>
            <th align='center'>Modelo: Regresión logística</th>
            <th colspan=4 align='center'>Métricas</tr>            
        </tr>
    </thead>
    <thead>
        <tr>
            <th>Clase</th>
            <th>Precision</th>
            <th>Recall</th>
            <th>F1</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td align="center">No</td>
            <td  align="center">0.90</td>
            <td  align="center">0.75</td>
            <td  align="center">0.82</td>                        
        </tr>
        <tr>
            <td align="center">Si</td>
            <td  align="center">0.52</td>
            <td  align="center">0.77</td>
            <td  align="center">0.62</td>            
        </tr>        
    </tbody>
</table>

Los resultados son similares a los obtenidos para el árbol de decisión. La métrica precisión es ligeramente superior con respecto a la misma métrica para el árbol. Lo mismo sucede con la métrica de sensibilidad.

La matriz de confusión del algoritmo de clasificación regresión repite lo mismo que lo mencionado para el modelo árbol. Esto significa que encontramos una alta tasa de verdadero positivos(251) y una baja tasa de falso negativos(76), lo cual se refleja en una  métrica alta de recall(0.77). Además, tenemos una alta tasa de falsos positivos, lo cual justifica que la precisión sea del 52%.

<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/matriz_regresion.png'>

Las curvas ROC y precisión x sesibilidad, repiten lo mismo que para las curvas del modelo árbol.

<div style="display:flex">
<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/auc_regresion.png'>
<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/pr_arbol.png'>

</div>

La validación cruzada arroja resultados similares en cuanto a la métrica precisión con respecto a la validación realizada sobre un solo conjunto de datos. Por ejemplo, el modelo árbol de decisión tuvo un una media de precisión de 0.52 ± 0.021, mientras que la regresión logística tuvo un promedio de precisión de  0.51 ± 0.013. No obstante, la puntuación de sensibilidad mejoró para ambos modelos. Dado que el algoritmo de clasificación árbol obtuvo un promedio de 0.76 ± 0.023 de sensibilidad y la regresión  obtuvo una media de sensibilidad de 0.80 ± 0.012.

### <strong>_Evaluación con datos de prueba_</strong>

El rendimiento de los modelos con los datos de prueba es similar al rendimiento obtenido en la validación cruzada. La precisión para los dos modelos se encuentra alrededor del 50% y la sensibilidad para el árbol de decisión es 0.77 y 0.80 para el modelo de regresión logística.

### <strong>_Análisis de importancia de las variables_</strong>

El modelo de regresión que calcula la probabilidad de  evasión de clientes queda descripto por la siguiente ecuación:

$P(Y=1|X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1X_1 + \beta_2X_2 + \beta_1X_3+ \beta_1X_4 + \beta_1X_5 + \beta_1X_6 + \beta_7X_7 + \beta_8X_8)}}$

donde $\beta_0$ es el intercepto de la regresión y $\beta_1$, $\beta_2$, $\beta_3$, $\beta_4$ ... $\beta_8$  son los coeficientes de las variables predictoras.

En este caso, el modelo queda definido por las siguientes variables predictoras:

$X_1$: SeniorCitizen,<br>
$X_2$: Fiber optic,<br>
$X_3$: Contract_one-year, <br>
$X_4$: Contract_two-year, <br>
$X_5$: PaymentMethod credit card', <br>
$X_6$: PaymentMethod electronic check', <br>
$X_7$: PaymentMethod mailed check', <br>
$X_8$: Tenure

y los coeficientes para cada variable independiente son:

$\beta_1$ = 0.33, <br>
$\beta_2$ = 1.28, <br>
$\beta_3$ = -0.80, <br>
$\beta_4$ = -1.74, <br>
$\beta_5$ = -0.05, <br>
$\beta_6$ = 0.63, <br>
$\beta_7$ = -0.13, <br>
$\beta_8$ = -1.42, <br>

Si observamos los coeficientes de la regresión, vemos que las variables Tenure y Contrato de dos años son las que tienen más importancia. El árbol de decisión también le asigna importancia al contrato a dos años. No obstante, este modelo también le asigna más importancia a la variable  Contrato a un año a diferencia de la regresión.

<img src='https://raw.githubusercontent.com/JGarcia575/challenge_telecomX_parte_2/refs/heads/main/img/importancia_arbol.png'>

### \begin{align}
    \color{yellow}{\textbf{Conclusión}}
    \end{align}

Dado que las métricas generales de rendimiento, como el AUC y la precisión promedio, son similares para ambos modelos, y reconociendo que la precisión máxima alcanzable para este problema es del 52%, se priorizó la capacidad de detectar a los clientes en riesgo. Por lo tanto, se eligió el modelo de regresión logística por su mayor sensibilidad (recall) en la validación cruzada y en la evaluación con los datos de prueba.

La regresión logística es un modelo básico pero sencillo de entender ya que de sus coeficientes podemos extraer como influye cada variable independienta en la probabilidad de churn. La cancelación se relaciona de manera positiva: con la edad, con el servicio de fibra óptica y con el método de pago cheque electrónico. Mientras que se relaciona de manera negativa con: los tipos de contratos, con los métodos de pagos tarjeta de crédito y cheque por correo y , por último, con antiguedad.

Las variables más importantes y, por lo tanto más influyentes para el modelo, son: antiguedad y contrato a dos años, las cuales pueden convertirse en el foco de diseños de estrategias para retener los clientes en riesgo de cancelación.

Dado que el modelo es agresivo identificando clientes en riesgo pero con la contra de que de todos los usuarios clasificados como churn, solo la mitad realmente se fue, recomendamos que las estrategias de retención de clientes sean económicas para compensar la debilidad del modelo.